## Data Recommendations

* movie_budgets is superior to movie_gross

#### Questions

1. Talent-driven approach to successful filmmaking. First, identify which talent are creating profitable and highly-discussed films.
* Locate actors, writers, directors, and producers that produce a high average profit in their projects.
* Create short list of most successful talent.
2. What does a successful movie look like? Runtime, genre?
* Find correlation between runtime and genre with profit. 
* Are there genres that the chosen talent prefer?
3. What should the budget be?
* How much is budget correlated with profit?

In [1]:
import pandas as pd
import sqlite3

## Movie_Gross

In [5]:
pd.read_csv("../Data/bom.movie_gross.csv.gz")

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [7]:
gross = pd.read_csv("../Data/bom.movie_gross.csv.gz")
gross['studio'].value_counts().head(40)

IFC        166
Uni.       147
WB         140
Fox        136
Magn.      136
SPC        123
Sony       110
BV         106
LGF        103
Par.       101
Eros        89
Wein.       77
CL          74
Strand      68
FoxS        67
RAtt.       66
KL          62
Focus       60
WGUSA       58
CJ          56
MBox        54
UTV         50
A24         49
WB (NL)     45
FM          42
LG/S        41
Cohen       40
ORF         37
SGem        35
Rela.       35
FIP         26
STX         24
Gold.       24
GK          24
Osci.       23
TriS        23
RTWC        23
BST         22
MNE         22
EOne        21
Name: studio, dtype: int64

## IMDB SQL

In [2]:
con = sqlite3.connect('../Data/im.db')
cursor=con.cursor()

In [3]:
%%script sqlite3 ../Data/im.db --out tables
.tables
.quit

In [8]:
print(tables)

directors      movie_akas     movie_ratings  principals   
known_for      movie_basics   persons        writers      



In [9]:
con = sqlite3.connect('../Data/im.db')
cursor = con.cursor()
imdb_schema_df = pd.read_sql('''
SELECT 
    *
FROM
    sqlite_master
''', con)

imdb_schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


## Base Query

In [10]:
master_sql = pd.read_sql('''
SELECT primary_title, start_year, runtime_minutes, genres, averagerating, primary_name, category
FROM principals
JOIN persons
    USING(person_id)
JOIN movie_basics
    USING(movie_id)
JOIN movie_ratings
    USING(movie_id)
WHERE start_year > 2011
AND numvotes > 1000
''', con)

master_sql

,primary_title,start_year,runtime_minutes,genres,averagerating,primary_name,category
0,The Other Side of the Wind,2018,122.0,Drama,6.9,Orson Welles,director
1,The Other Side of the Wind,2018,122.0,Drama,6.9,Peter Bogdanovich,actor
2,The Other Side of the Wind,2018,122.0,Drama,6.9,John Huston,actor
3,The Other Side of the Wind,2018,122.0,Drama,6.9,Susan Strasberg,actress
4,The Other Side of the Wind,2018,122.0,Drama,6.9,Gary Graver,cinematographer
...,...,...,...,...,...,...,...
75590,The Hard Way,2019,92.0,Action,4.7,Michael Jai White,actor
75591,The Hard Way,2019,92.0,Action,4.7,Thomas J. Churchill,writer
75592,The Hard Way,2019,92.0,Action,4.7,Randy Couture,actor
75593,The Hard Way,2019,92.0,Action,4.7,Madalina Anea,actress


In [11]:
master_sql['primary_name'].value_counts().head(20)


Jason Blum              61
Mercedes Gamero         30
James Franco            29
David Michael Latt      29
Necati Akpinar          28
Tim Bevan               27
Bhushan Kumar           26
Randall Emmett          26
Nicolas Cage            25
Eric Fellner            25
Anurag Kashyap          25
Akshay Kumar            25
Gopi Sundar             24
Karan Johar             24
Samantha Ruth Prabhu    24
Siddharth Roy Kapur     23
John Cusack             23
Peter Safran            23
Alexandre Desplat       23
Mikel Lejarza           22
Name: primary_name, dtype: int64

## Movie Info

In [12]:
pd.read_csv('../Data/rt.movie_info.tsv.gz', sep='\t')


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


## Rotten Tomato Reviews

In [13]:
pd.read_csv('../Data/rt.reviews.tsv.gz', delimiter="\t", encoding = 'unicode_escape')


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


## The Movie Database

In [14]:
pd.read_csv('../Data/tmdb.movies.csv.gz')


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


## Budget Info

In [15]:
budget_info = pd.read_csv('../Data/tn.movie_budgets.csv.gz')
budget_info

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


## Merging SQL and Budget

In [16]:
relevant_budget_info = budget_info.copy()
relevant_budget_info.rename(columns = {'movie':'primary_title'}, inplace = True)
relevant_budget_info = relevant_budget_info[['primary_title', 'production_budget', 'worldwide_gross']]
relevant_budget_info

,primary_title,production_budget,worldwide_gross
0,Avatar,"$425,000,000","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875"
2,Dark Phoenix,"$350,000,000","$149,762,350"
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$1,316,721,747"
...,...,...,...
5777,Red 11,"$7,000",$0
5778,Following,"$6,000","$240,495"
5779,Return to the Land of Wonders,"$5,000","$1,338"
5780,A Plague So Pleasant,"$1,400",$0


In [17]:
joined_df = pd.merge(master_sql, relevant_budget_info, on=['primary_title'])
joined_df['primary_title'].value_counts()

Legend                   40
Life                     40
The Square               40
Beauty and the Beast     40
The Gambler              40
                         ..
Life of the Party         9
Under the Rainbow         8
Crowsnest                 8
Indie Game: The Movie     7
Hawaii                    7
Name: primary_title, Length: 1269, dtype: int64

In [18]:
joined_df

,primary_title,start_year,runtime_minutes,genres,averagerating,primary_name,category,production_budget,worldwide_gross
0,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Charlie Sheen,actor,"$45,000,000","$73,706"
1,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Sean Catherine Derek,writer,"$45,000,000","$73,706"
2,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Haylie Duff,actress,"$45,000,000","$73,706"
3,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Hilary Duff,actress,"$45,000,000","$73,706"
4,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Robert Engelman,producer,"$45,000,000","$73,706"
...,...,...,...,...,...,...,...,...,...
13428,Unplanned,2019,106.0,"Biography,Drama",6.3,Chris Jones,producer,"$6,000,000","$18,107,621"
13429,Unplanned,2019,106.0,"Biography,Drama",6.3,Ashley Bratcher,actress,"$6,000,000","$18,107,621"
13430,Unplanned,2019,106.0,"Biography,Drama",6.3,Jared Lotz,actor,"$6,000,000","$18,107,621"
13431,Unplanned,2019,106.0,"Biography,Drama",6.3,Brooks Ryan,actor,"$6,000,000","$18,107,621"


In [19]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13433 entries, 0 to 13432
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   primary_title      13433 non-null  object 
 1   start_year         13433 non-null  int64  
 2   runtime_minutes    13433 non-null  float64
 3   genres             13433 non-null  object 
 4   averagerating      13433 non-null  float64
 5   primary_name       13433 non-null  object 
 6   category           13433 non-null  object 
 7   production_budget  13433 non-null  object 
 8   worldwide_gross    13433 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.0+ MB


## Remove commas and dollar signs

In [20]:
joined_df['production_budget'] = joined_df['production_budget'].str.replace(',', '').str.replace('$', '').astype(int)
joined_df['worldwide_gross'] = joined_df['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(int)

joined_df

,primary_title,start_year,runtime_minutes,genres,averagerating,primary_name,category,production_budget,worldwide_gross
0,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Charlie Sheen,actor,45000000,73706
1,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Sean Catherine Derek,writer,45000000,73706
2,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Haylie Duff,actress,45000000,73706
3,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Hilary Duff,actress,45000000,73706
4,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Robert Engelman,producer,45000000,73706
...,...,...,...,...,...,...,...,...,...
13428,Unplanned,2019,106.0,"Biography,Drama",6.3,Chris Jones,producer,6000000,18107621
13429,Unplanned,2019,106.0,"Biography,Drama",6.3,Ashley Bratcher,actress,6000000,18107621
13430,Unplanned,2019,106.0,"Biography,Drama",6.3,Jared Lotz,actor,6000000,18107621
13431,Unplanned,2019,106.0,"Biography,Drama",6.3,Brooks Ryan,actor,6000000,18107621


## Create profit column

In [21]:
joined_df['profit'] = joined_df['worldwide_gross'] - joined_df['production_budget']
joined_df

,primary_title,start_year,runtime_minutes,genres,averagerating,primary_name,category,production_budget,worldwide_gross,profit
0,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Charlie Sheen,actor,45000000,73706,-44926294
1,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Sean Catherine Derek,writer,45000000,73706,-44926294
2,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Haylie Duff,actress,45000000,73706,-44926294
3,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Hilary Duff,actress,45000000,73706,-44926294
4,Foodfight!,2012,91.0,"Action,Animation,Comedy",1.9,Robert Engelman,producer,45000000,73706,-44926294
...,...,...,...,...,...,...,...,...,...,...
13428,Unplanned,2019,106.0,"Biography,Drama",6.3,Chris Jones,producer,6000000,18107621,12107621
13429,Unplanned,2019,106.0,"Biography,Drama",6.3,Ashley Bratcher,actress,6000000,18107621,12107621
13430,Unplanned,2019,106.0,"Biography,Drama",6.3,Jared Lotz,actor,6000000,18107621,12107621
13431,Unplanned,2019,106.0,"Biography,Drama",6.3,Brooks Ryan,actor,6000000,18107621,12107621


## Finding average profit sorted by talent

In [22]:
joined_df2 = joined_df.sort_values(by='profit', ascending=False)

In [23]:
joined_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13433 entries, 11204 to 13053
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   primary_title      13433 non-null  object 
 1   start_year         13433 non-null  int64  
 2   runtime_minutes    13433 non-null  float64
 3   genres             13433 non-null  object 
 4   averagerating      13433 non-null  float64
 5   primary_name       13433 non-null  object 
 6   category           13433 non-null  object 
 7   production_budget  13433 non-null  int32  
 8   worldwide_gross    13433 non-null  int32  
 9   profit             13433 non-null  int32  
dtypes: float64(2), int32(3), int64(1), object(4)
memory usage: 997.0+ KB


In [24]:
joined_df3 = joined_df2.groupby('primary_name', as_index=False)['profit'].mean()
joined_df4 = joined_df3.sort_values(by='profit', ascending=False)
joined_df4.head(20)

,primary_name,profit
7600,Ty Simpkins,1.433855e+09
5289,Michael Crichton,1.284814e+09
2603,Gary Scott Thompson,1.156785e+09
3643,Joe Robert Cole,1.148258e+09
6169,Rafe Spall,1.135773e+09
4411,Kristen Anderson-Lopez,1.122470e+09
2846,Hans Christian Andersen,1.122470e+09
2992,Idina Menzel,1.122470e+09
3879,Jonathan Groff,1.122470e+09
1317,Chris Buck,1.122470e+09
